In [ ]:
"""
x1. read in dyndb spreadsheet
x2. read in static database
x3. assign PNs to dyndb

4. instantiate and load weights for multi-param [3D]CNN
[4a. figure out how to get embedding vector... look @ your '9_test_troubleshoot_experiment/vizualize_3DCNN_filters.ipynb']
5. load, run each topo array through 3DCNN --> extract predicted properties and feature vector
    ???? predicted properties in a single vector in a single column, or separate columns?... probably separate, so I can pick and choose...

6... Profit? I think that's it...



"""

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objs as go
from plotly.subplots import make_subplots



In [ ]:
dyndb

In [ ]:
dyndb.columns.values

In [ ]:
nbpath     = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN'
voxel_path = '/home/mgolub4/DLproj/MLTO_2024/0_data/voxel_arrays_npy_by_partno'

# dbpath = os.path.join(nbpath, 'dyn_data', 'dynamic_static_database_scaled_DEC23_constit_eqn_fit_params.csv')
dbpath = os.path.join(nbpath, 'dyn_data', 'dynamic_static_database_scaled_APR24_constit_eqn_fit_params.csv')


dyndb = pd.read_csv(dbpath)

stress_ser_path = os.path.join(nbpath, 'dyn_data/stress_series_data')

csvs = [p for p in sorted(os.listdir(stress_ser_path)) if p.endswith('untrunc.csv')]


In [ ]:
# Add Part Number column to 
dyndb['full PN'] = pd.Series()

In [ ]:
dyndb_csvs = (dyndb.dyn_file_name_original + '_processed_untrunc').values

In [ ]:
statdb = pd.read_csv('/home/mgolub4/DLproj/MLTO_2024/0_data/data_3phys_w_dyn_topos_scaled_PNs.csv')

In [ ]:
# dyndb.head()

In [ ]:
# statdb.head()

In [ ]:
i = 12

cell_type = dyndb.loc[i, 'cell_type']
volfrac = dyndb.loc[i, 'volFrac']

In [ ]:
statdb[(statdb.cell_type == cell_type) & (statdb.volFrac == volfrac)]['full PN'].values[0]

In [ ]:
for i in dyndb.index:
    cell_type = dyndb.loc[i, 'cell_type']
    volfrac = dyndb.loc[i, 'volFrac']

    partnum = statdb[(statdb.cell_type == cell_type) & (statdb.volFrac == volfrac)]['full PN'].values[0]

    dyndb.loc[i, 'full PN'] = partnum



In [ ]:
# dyndb.full_PN

In [ ]:
from v4_multimod_attn_8mlp_wRelu_longer_featvec import Linear_Feature_Attention, CNN3D_multimod_attn_v4

In [ ]:
cp_dir = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dataset_setup/CP_run18_sd17_augm_vf0p98_v4_multimod_attn_8mlp_wRelu_longer_featvec_28par_allphys.pth'

In [ ]:
params = ['volFrac', 
        'CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',
        'CH_12 scaled', 'CH_13 scaled','CH_23 scaled',
        'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
        'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
        'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
        'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
        'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

params_segmented = [['volFrac',], 
                    ['CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',],
                    ['CH_12 scaled', 'CH_13 scaled','CH_23 scaled',],
                    ['EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',],
                    ['GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled',],
                    ['vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',],
                    ['KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled',],
                    ['kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F


import torch.utils.data as data
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
cnn = CNN3D_multimod_attn_v4(params_segmented).to(device)

In [ ]:
weights = torch.load(cp_dir)

In [ ]:
from collections import OrderedDict

In [ ]:
non_module_weights = OrderedDict()

for k, v in weights.items():
    name = k[7:] # remove 'module.'
    non_module_weights[name] = v

In [ ]:
cnn.load_state_dict(non_module_weights)

In [ ]:
cnn.eval()

In [ ]:
dyndb['conv_feat_vec'] = pd.Series()
pred_param_cols = []
for par in params:
    colname = f'pred {par}'
    dyndb[colname] = pd.Series()
    pred_param_cols.append(colname)

In [ ]:
partnum = dyndb.loc[0, 'full PN']
arrpath = os.path.join(voxel_path, f'{partnum}.npy')
arr = np.expand_dims(np.load(arrpath), axis=(0,1)).astype(np.float32)
arr_tensor = torch.from_numpy(arr)

In [ ]:
arr_tensor = arr_tensor.to(device)


In [ ]:
def embed_vector(in_tensor, cnn):
    in_tensor = in_tensor.to(device)
    for i in range(len(cnn.convolutions)):
        layer_out = cnn.convolutions[i](in_tensor)

        layer = cnn.convolutions[i]

        in_tensor = layer_out

    in_tensor = in_tensor.detach().cpu().numpy()
        
    in_tensor = np.squeeze(in_tensor, axis=(2,3,4))
    return in_tensor

In [ ]:
arrt = embed_vector(arr_tensor, cnn)

In [ ]:
cnn(arr_tensor.to(device)).detach().cpu().numpy()

In [ ]:
conv_feature_vector

In [ ]:
featmap_folder = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/voxel_embedding_feature_maps'

In [ ]:
for i in dyndb.index[:]:
    partnum = dyndb.loc[i, 'full PN']
    arrpath = os.path.join(voxel_path, f'{partnum}.npy')
    arr = np.expand_dims(np.load(arrpath), axis=(0,1)).astype(np.float32)
    arr_tensor = torch.from_numpy(arr)

    conv_feature_vector = embed_vector(arr_tensor, cnn)

    emb_vec_partnum = partnum + '_emb_vec'
    emb_vec_fpath = os.path.join(featmap_folder, emb_vec_partnum)
    np.save(emb_vec_fpath, conv_feature_vector)
    
    param_vec = cnn(arr_tensor.to(device)).detach().cpu().numpy()

    dyndb.loc[i, 'conv_feat_vec'] = emb_vec_partnum

    for j, col in enumerate(pred_param_cols):
        dyndb.loc[i, col] = param_vec[0,j]

    

In [ ]:
dyndb.to_csv('/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/dyn_stat_database_PINN_ready.csv')